# Can we use ground sensors to evaluate London residence's exposure to high air pollution?
categories: [data wrangling, data exploration]

### Author
- Christina Last. [The Alan Turing Institute](https://www.turing.ac.uk/people/research-engineering/christina-last).
GitHub: [@ChristinaLast](https://github.com/ChristinaLast/) Twitter: [@last_christina](https://twitter.com/last_christina)

All the authors above contributed to this data story story with ideas, code development and story telling.

### Reviewers
- [insert name here]

## London air quality
Some of the main drivers of climate change result in poor air quality in urban areas. While climate change is abstract and global, air quality is local and personal:
a few years back, poor air quality was reported outside thousands of schools in the UK a recent legal ruling determined that air pollution was the cause of death for an 8-year-old London girl
the Mayor of London has plans to reduce air pollution at schools

The story reveals which communities are most impacted by air pollution in London. It also explories the risk of walking to school by different routes, and possibly looking over time at locations where school street schemes have been implemented.

### Downloading the air quality data locally
This Turing Data Story uses data from [Global Clean Air](https://www.globalcleanair.org/innovative-air-quality-monitoring/london-uk/breathe-london-data/).
The Data is available to download from the link.

### Analysis Outline

#### :wrench: Setup

We begin by setting up our environment and importing various python libraries that we will be using for the analysis.
In particular, `pandas` and `numpy` are key data science libraries used for data processing. `matplotlib` and `seaborn` will help us visualise our data.

## :bar_chart: Data Exploration
We are using data from [Global Clean Air](https://www.globalcleanair.org/innovative-air-quality-monitoring/london-uk/breathe-london-data/)
which contains information about pollution’s health effects, as well as readily available — and understandable — air pollution data and analysis.
The Breathe London pilot project mapped and measured pollution across the capital, led for two years by Environmental Defense Fund Europe and launched in partnership with the Mayor of London and leading science and technology experts.

The dataset contains more than 100 lower-cost sensor pods and specially-equipped Google Street View cars,
Breathe London complemented and expanded upon London’s existing monitoring networks.
The project aimed to help people better understand their local air quality and support cities around the world with future monitoring initiatives.

In [21]:
from pathlib import Path
# reading in the data
import pandas as pd

root_data_path = Path("data")

def read_local_data(data_dir, filename):
    data_path = Path(str(data_dir)+filename).absolute()
    return pd.read_csv(data_path)



pm25_metadata = read_local_data(root_data_path, "/PM2_5_site_metadata_20180901_20201130.csv")
pm25_full_network = read_local_data(root_data_path, "/20180901_20201130_PM2_5_AQMesh_Scaled_Dataset_UGM3.csv")
no2_metadata = read_local_data(root_data_path, "/NO2_site_metadata_20180901_20201130.csv")
no2_full_network = read_local_data(root_data_path, "/20180901_20201130_NO2_AQMesh_Scaled_Dataset_UGM3.csv")

In [22]:
pm25_metadata.head()

,pod_id_location,start_date_UTC,end_date_UTC,relocate_date_UTC,location_name,Borough,Type,ULEZ,Latitude,Longitude,X_Coord,Y_Coord,Distance_from_Road,Height,Site_Type
0,1245,15/02/2019 12:00,30/11/2020 23:59,NaN,Dollis Road,Barnet,Kerbside,No,51.604486,-0.206551,524299.9229,191038.6267,1.2,3.100,NaN
1,2245,03/05/2019 10:00,30/11/2020 23:59,NaN,Great Ormond St Hospital,Camden,Roadside,Yes,51.521880,-0.120434,530499.4261,182002.3181,2.5,2.800,Hospital
2,3245,04/10/2018 18:00,30/11/2020 23:59,NaN,Savernake Road,Camden,Urban Background,No,51.555485,-0.152338,528191.7240,185682.9587,5.0,2.297,School
3,4245,06/03/2019 14:35,30/11/2020 23:59,NaN,Wandsworth Road,Lambeth,Kerbside,No,51.475735,-0.131209,529881.9791,176852.2693,0.5,3.120,NaN
4,5245,06/03/2019 15:55,30/11/2020 23:59,NaN,Garrett Lane,Wandsworth,Kerbside,No,51.440399,-0.186775,526117.9944,172825.4459,1.0,3.445,Individual


In [23]:
# And displaying the data itself
pm25_full_network.head()

,date_UTC,pm2_5_ugm3,pod_id_location,pod_id,location_name,ratification_status
0,2019-02-15 13:00:00,20.17,1245,1245,Dollis Road,P
1,2019-02-15 14:00:00,18.47,1245,1245,Dollis Road,P
2,2019-02-15 15:00:00,19.84,1245,1245,Dollis Road,P
3,2019-02-15 16:00:00,36.68,1245,1245,Dollis Road,P
4,2019-02-15 17:00:00,20.18,1245,1245,Dollis Road,P


We can see that there is a csv recording the timeseries value of the PM2.5 reading (hourly) and a csv containing information
about the location of the sensor. They both share a commin id in the column `pod_id_location`. Lets join the PM2.5 data and
the NO2 data to one dataframe containing the timeseries values and location information

In [24]:
def join_network_metadata(network_df, metadata_df, col_to_join):
    return network_df.merge(metadata_df, on=col_to_join, how='outer', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


pm25_network_metadata = join_network_metadata(pm25_full_network, pm25_metadata,  "pod_id_location")
no2_network_metadata = join_network_metadata(no2_full_network, no2_metadata,  "pod_id_location")
no2_network_metadata.head()

,date_UTC,no2_ugm3,pod_id_location,pod_id,location_name,ratification_status,start_date_UTC,end_date_UTC,relocate_date_UTC,Borough,Type,ULEZ,Latitude,Longitude,X_Coord,Y_Coord,Distance_from_Road,Height,Site_Type
0,2019-02-15 17:00:00,61.99,1245,1245,Dollis Road,P,15/02/2019 16:45,30/11/2020 23:59,NaN,Barnet,Kerbside,No,51.604486,-0.206551,524299.9229,191038.6267,1.2,3.1,NaN
1,2019-02-15 18:00:00,67.33,1245,1245,Dollis Road,P,15/02/2019 16:45,30/11/2020 23:59,NaN,Barnet,Kerbside,No,51.604486,-0.206551,524299.9229,191038.6267,1.2,3.1,NaN
2,2019-02-15 19:00:00,72.61,1245,1245,Dollis Road,P,15/02/2019 16:45,30/11/2020 23:59,NaN,Barnet,Kerbside,No,51.604486,-0.206551,524299.9229,191038.6267,1.2,3.1,NaN
3,2019-02-15 20:00:00,67.24,1245,1245,Dollis Road,P,15/02/2019 16:45,30/11/2020 23:59,NaN,Barnet,Kerbside,No,51.604486,-0.206551,524299.9229,191038.6267,1.2,3.1,NaN
4,2019-02-15 21:00:00,59.78,1245,1245,Dollis Road,P,15/02/2019 16:45,30/11/2020 23:59,NaN,Barnet,Kerbside,No,51.604486,-0.206551,524299.9229,191038.6267,1.2,3.1,NaN


Here we can see the NO2 data has successfully been joined with the metadata available for the sensor location. We drop repeated columns
using a regex filter on our suffix `_DROP` added to duplicated columns within the merge.

In [25]:
no2_numeric_cols = ["no2_ugm3", "Distance_from_Road","Height"]
pm25_numeric_cols = ["pm2_5_ugm3", "Distance_from_Road","Height"]

In [35]:
# Data distribution
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pm25_network_metadata[pm25_numeric_cols].describe()

,pm2_5_ugm3,Distance_from_Road,Height
count,1399810.00000,1305125.00000,1399810.00000
mean,-104.43024,4.61179,3.27866
std,321.54480,6.96932,1.95759
min,-999.00000,0.00000,1.50000
25%,4.28000,1.00000,2.90000
50%,6.40000,2.00000,3.00000
75%,11.20000,5.70000,3.12600
max,469.59000,50.00000,19.50000


In [36]:
no2_network_metadata[no2_numeric_cols].describe()

,no2_ugm3,Distance_from_Road,Height
count,1555296.00000,1448871.00000,1555296.00000
mean,-15.62420,4.18273,3.29576
std,223.95386,6.57160,1.80931
min,-999.00000,0.00000,1.50000
25%,22.28000,1.00000,2.93000
50%,30.02000,1.67000,3.00000
75%,40.85000,5.00000,3.12000
max,628.12000,50.00000,20.00000


The distribution of the numberic variables looks

In [32]:
#nan values
pm25_network_metadata.isnull().sum()

date_UTC                     0
pm2_5_ugm3                   0
pod_id_location              0
pod_id                       0
location_name                0
ratification_status          0
start_date_UTC               0
end_date_UTC                 0
relocate_date_UTC      1344809
Borough                      0
Type                         0
ULEZ                         0
Latitude                     0
Longitude                    0
X_Coord                      0
Y_Coord                      0
Distance_from_Road       94685
Height                       0
Site_Type               684790
dtype: int64

In [31]:
len(pm25_network_metadata_nona)

19

In [29]:
#no. of unique values
pm25_network_metadata_nona.sort_values()[:10]

ratification_status     1
ULEZ                    2
Type                    4
Site_Type               5
relocate_date_UTC       6
end_date_UTC            6
Borough                29
Height                 49
Distance_from_Road     52
pod_id                 95
dtype: int64